In [1]:
from IPython.display import display, Markdown, Latex
import pandas as pd
import os
import yaml
import requests
import folium
import EDA.map_functions as maps
import geopandas as gpd
import walk_score
from geopy.geocoders import Nominatim
from shapely.geometry import Point, Polygon
from EDA.utils import get_SODA_data

In [2]:
ROOT = "C:/Users/Beau/Documents/GitHub/RealEstate"
SHAPE_FILES = os.path.join(ROOT, "data", "raw", "shape_files")
RAW = os.path.join(ROOT, "data", "raw")
CLEANED = os.path.join(ROOT, "data", "processed")

In [3]:
# load location data
df_location_combined_raw = pd.read_pickle(
    os.path.join(CLEANED, "whats_nearby_location_data.pkl")
)
df_metra = pd.read_csv(os.path.join(RAW, "Metra_Stations.csv"))
df_location_combined = pd.concat([df_location_combined_raw, df_metra])
df_hospitals = gpd.read_file(os.path.join(SHAPE_FILES, "Hospitals.zip"))

# load shape data
df_shape_data_combined = pd.read_pickle(
    os.path.join(CLEANED, "whats_nearby_shape_data.pkl")
)

# Enter address here:

In [4]:
address = "5024 W School St Chicago, IL 60641"

In [5]:
# get geographic coordinates
geolocator = Nominatim(user_agent="beau.h.smit@gmail.com")
location = geolocator.geocode(address)
LAT, LON = location.latitude, location.longitude
property_coordinates = gpd.GeoDataFrame(
    {"geometry": [Point(LON, LAT)]}, crs="EPSG:4326"
)
print(f"coordinates: {(LAT, LON)}")

coordinates: (41.940660485930934, -87.7525329065122)


In [6]:
# get walk scores
with open(os.path.join(ROOT, "credentials.yml"), mode="r") as file:
    wskey = yaml.safe_load(file)["walk-score-key"]

# make request
# walk, transit, bike = walk_score.get_walk_score_from_address(address, wskey)
walk, transit, bike = walk_score.get_walk_score_from_coord(LAT, LON, wskey)

In [7]:
# exclude locations far from property
lat_min = LAT - 0.03
lat_max = LAT + 0.03
lon_min = LON - 0.04
lon_max = LON + 0.04

In [8]:
# limit the markers on the map
df_location_map = df_location_combined.loc[
    df_location_combined.source.isin(
        [
            "liquor",
            "divvy",
            "murals",
            "EV_chargers",
            "landmarks",
            "L",
            "grocery",
            "park_art",
            "farmers_market",
            "hospitals",
        ]
    )
    & (df_location_combined.latitude.between(lat_min, lat_max))
    & (df_location_combined.longitude.between(lon_min, lon_max)),
    :,
]

In [9]:
# add shape/boundary data
# https://leafletjs.com/reference.html#path-option

# add mobility areas to the map
# mobility_area_style = lambda feature: {
#     'color': 'lightgreen',
#     'fill_color': 'lightgreen',
#     'weight': 2
# }
# folium.GeoJson(data=df_mobility_areas["geometry"], style_function=mobility_area_style).add_to(m)

# add ADU areas to the map
# ADU_area_style = lambda feature: {
#     'color': 'lightblue',
#     'fill_color': 'lightblue',
#     'weight': 2
# }
# folium.GeoJson(data=df_ADU["geometry"], style_function=ADU_area_style).add_to(m)

In [10]:
# add bike lanes to the map
# bike_lane_style = lambda feature: {
#     'color': 'green',
#     'weight': 4
# }
# folium.GeoJson(
#     data=df_shape_data_combined.loc[df_shape_data_combined.source == "bike_routes", "geometry"],
#     style_function=bike_lane_style
# ).add_to(m)

In [11]:
# icons: https://fontawesome.com/search
# color choices: ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred', 'beige', 'darkblue',
# 'darkgreen', 'cadetblue', 'darkpurple', 'white', 'pink', 'lightblue', 'lightgreen', 'gray', 'black', 'lightgray']

# place_markers(df_metra, label_col='STATION_NAME', color='lightblue', icon='train-subway')
# licenses            920294
# permits             726509
# current_licenses     50607
source_dict = {
    "liquor": {"name_column": "", "color": "orange", "icon": "martini-glass"},
    "divvy": {"name_column": "", "color": "blue", "icon": "bicycle"},
    "murals": {"name_column": "", "color": "pink", "icon": "palette"},
    "EV_chargers": {"name_column": "", "color": "darkgreen", "icon": "bolt"},
    "landmarks": {"name_column": "", "color": "gray", "icon": "landmark"},
    "L": {"name_column": "", "color": "darkblue", "icon": "train-subway"},
    "grocery": {"name_column": "", "color": "green", "icon": "cart-shopping"},
    "park_art": {"name_column": "", "color": "lightred", "icon": "bench-tree"},
    "farmers_market": {"name_column": "", "color": "lightgreen", "icon": "seedling"},
}

In [12]:
# Create a Map instance for Chicago
m = folium.Map(location=[LAT, LON], zoom_start=16)

# add marker for the property of interest
maps.add_map_marker(
    m, lat=LAT, lon=LON, name="property", color="black", icon="house-chimney"
).add_to(m)

# add circles for distance reference around property
maps.add_map_circle(
    m, lat=LAT, lon=LON, radius=800
)  # 800 meters = 0.5 miles; about a 12 minute walk
maps.add_map_circle(
    m, lat=LAT, lon=LON, radius=1600
)  # 800 meters = 1 miles; about a 25 minute walk


# hospitals
folium.GeoJson(
    data=df_hospitals["geometry"],
    marker=folium.Marker(icon=folium.Icon(color="red", icon="circle-h", prefix="fa")),
).add_to(m)

feature_group = folium.FeatureGroup("Locations")

for idx, row in df_location_map.iterrows():
    feature_group.add_child(
        maps.add_map_marker(
            m,
            lat=row["latitude"],
            lon=row["longitude"],
            name=row["label"],
            color=source_dict[row["source"]]["color"],
            icon=source_dict[row["source"]]["icon"],
        )
    )

# Change the map style
# m.add_tile_layer(tiles='Stamen Toner', name='Stamen Toner')
# map.save(outfile = "test.html")
m.add_child(feature_group)

In [13]:
# what is the zoning?
zoning = gpd.sjoin(
    property_coordinates,
    gpd.GeoDataFrame(
        df_shape_data_combined.loc[df_shape_data_combined.source == "zoning"]
    ),
)["label"][0]

# which ward is it in?
ward = gpd.sjoin(
    property_coordinates,
    gpd.GeoDataFrame(
        df_shape_data_combined.loc[df_shape_data_combined.source == "ward"]
    ),
)["label"][0]

# which neighborhood is it in?
neighborhood = gpd.sjoin(
    property_coordinates,
    gpd.GeoDataFrame(
        df_shape_data_combined.loc[df_shape_data_combined.source == "neighborhood"]
    ),
)["label"][0]

# which school district is it in?
hs = gpd.sjoin(
    property_coordinates,
    gpd.GeoDataFrame(
        df_shape_data_combined.loc[df_shape_data_combined.source == "high_schools"]
    ),
)["label"][0]

# is this property in ADU area?
adu_ind = (
    gpd.sjoin(
        property_coordinates,
        gpd.GeoDataFrame(
            df_shape_data_combined.loc[df_shape_data_combined.source == "ADU"]
        ),
    ).shape[0]
    > 0
)

# is this property in a mobility zone?
mobility_ind = (
    gpd.sjoin(
        property_coordinates,
        gpd.GeoDataFrame(
            df_shape_data_combined.loc[
                df_shape_data_combined.source == "mobility_areas"
            ]
        ),
    ).shape[0]
    > 0
)

# is this property in an enterprise zone?
enterprise_ind = (
    gpd.sjoin(
        property_coordinates,
        gpd.GeoDataFrame(
            df_shape_data_combined.loc[
                df_shape_data_combined.source == "enterprise_zone"
            ]
        ),
    ).shape[0]
    > 0
)

In [14]:
display(
    Markdown(
        f"\
# Address: {address}\n\
## Zoning: {zoning} https://secondcityzoning.org/zones/\n\
### walk score: {walk}\n\
### transit score: {transit}\n\
### bike score: {bike}\n\
## ADU Area: {adu_ind}\n\
## Mobility Area: {mobility_ind}\n\
## Neighborhood: {neighborhood}\n\
### High School: {hs}\n\
### Ward: {ward}\n\
"
    )
)
### Enterprise Zone: {enterprise_ind}\n\

# Address: 5024 W School St Chicago, IL 60641
## Zoning: RS-3 https://secondcityzoning.org/zones/
### walk score: 82
### transit score: None
### bike score: 58
## ADU Area: False
## Mobility Area: True
## Neighborhood: Portage Park
### High School: FOREMAN HS
### Ward: 31


In [15]:
# Question: if a property is zoned as RS-3, how are there multiple units?
# Is that okay? Did they have to get a permit? Was it grandfathered in?